Pour un bon fonctionnement, il est impératif d'utiliser une version d'opencv-contrib inférieure à 4.7, comme par exemple opencv-contrib-python-4.6.0.66.

Sinon, le Charuco board ne sera pas détecté en raison de modifications et corrections en cours dans l'API d'OpenCV.

In [16]:
import numpy as np
import cv2
from cv2 import aruco
import glob

Importer les images des caméras

In [17]:
images_calibration_b1 = glob.glob('B1/Calibration_B1/Intrinsic/*.tiff')
images_calibration_b2 = glob.glob('B2/Calibration_B2/Intrinsic/*.tiff')
images_calibration_b3 = glob.glob('B3/Calibration_B3/Intrinsic/*.tiff')

images_b1 = glob.glob('B1/GM8_B1/*.tiff')
images_b2 = glob.glob('B2/GM8_B2/*.tiff')
images_b3 = glob.glob('B3/GM8_B3/*.tiff')

path_save = 'results/'

Calibration intrinsèque qui renvoie la précision, la matrice intrinsèque (fx et fy), les coefficients de distorsion de la lentille, vecteurs de rotation et vecteurs de translation

In [18]:
def calibrate_camera(images):
    
    # Variables pour le tableau ChAruco
    CHARUCOBOARD_ROWCOUNT = 6
    CHARUCOBOARD_COLCOUNT = 10
    ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_1000)
    
    # Créer les constantes à passer aux méthodes Aruco
    CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=CHARUCOBOARD_COLCOUNT,
        squaresY=CHARUCOBOARD_ROWCOUNT,
        squareLength=0.04,
        markerLength=0.02,
        dictionary=ARUCO_DICT
    )
    
     # Créer les tableaux et variables pour stocker les informations telles que les coins et les IDs des images traitées
    corners_all = []  # Coins détectés dans toutes les images traitées
    ids_all = [] # IDs Aruco correspondants aux coins détectés
    image_size = None # Déterminé à l'exécution

    # Boucler à travers les images
    for iname in images:
        img = cv2.imread(iname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Trouver les marqueurs Aruco dans l'image
        corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT
        )

        # Dessiner les marqueurs Aruco trouvés dans l'image
        img = aruco.drawDetectedMarkers(
            image=img, 
            corners=corners
        )

        # Obtenir les coins et IDs Charuco à partir des marqueurs Aruco détectés
        response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD
        )

        # Si un tableau Charuco a été trouvé, collecter les points d'image/coin
        # Nécessitant au moins 20 carré
        if response > 20:
            
            # Ajouter ces coins et IDs aux tableaux de calibration
            corners_all.append(charuco_corners)
            ids_all.append(charuco_ids)

            # Dessiner le tableau Charuco détecté pour montrer que le tableau a été correctement détecté
            img = aruco.drawDetectedCornersCharuco(
                    image=img,
                    charucoCorners=charuco_corners,
                    charucoIds=charuco_ids)
            
            # Si la taille de l'image est inconnue, la définir maintenant
            if not image_size:
                image_size = gray.shape[::-1]
                
            # Redimensionner l'image en proportion, en limitant la largeur ou la hauteur à 1000
            proportion = max(img.shape) / 1000.0
            img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
            
            # Pause pour afficher chaque image, en attendant une pression de touche
            cv2.imshow('Charuco board', img)
            cv2.waitKey(0)
        else:
            print("Impossible de détecter un tableau Charuco dans l'image : {}".format(iname))

    cv2.destroyAllWindows()

    # Vérifier qu'au moins une image a été trouvée
    if len(images) < 1:
        # Calibration échouée car aucune image n'a été trouvée
        print("La calibration a échoué. Aucune image de tableaux Charuco n'a été trouvée. Ajoutez des images de tableaux Charuco et utilisez ou modifiez les conventions de nommage utilisées dans ce fichier.")
        exit()

    # Vérifier si la taille de l'image a été déterminée
    if not image_size:
        # Calibration échouée car aucun tableau Charuco n'a été détecté
        print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
        exit()

    # Maintenant que toutes les images ont été traitées, effectuer la calibration de la caméra basée sur l'ensemble des points découverts
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)
    return ret, mtx, dist, rvecs, tvecs

# Calibration des caméras
matrix_b1 = calibrate_camera(images_calibration_b1)
matrix_b2 = calibrate_camera(images_calibration_b2)
matrix_b3 = calibrate_camera(images_calibration_b3)
        

Corriger la distorsion des images choisies

In [19]:
def undistort(images, mtx, dist):
    # Créer une liste pour stocker les images non déformées
    all_undistorted_images = []
    for iname in images:
        img = cv2.imread(iname)
        img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Appliquer la correction de distorsion
        undistorted_img = cv2.undistort(gray, mtx, dist)
        all_undistorted_images.append(undistorted_img)
        
    return all_undistorted_images

# Correction de distorsion des images

undistorted_images_b1 = undistort(images_b1, matrix_b1[1], matrix_b1[2])
undistorted_images_b2 = undistort(images_b2, matrix_b2[1], matrix_b2[2])
undistorted_images_b3 = undistort(images_b3, matrix_b3[1], matrix_b3[2])

Détecter et dessiner les régions d'intérêt maximales stables (MSER) et sélectionner celles avec une circularité suffisante sur l'image "img"

In [20]:
# Sélectionner ici une image pour la détection des régions MSER
# Vous pouvez changer l'index pour sélectionner une autre image
# Par défaut, l'image 0 de B3 est sélectionnée

img = undistorted_images_b3[0]

# Créer un objet MSER
mser = cv2.MSER_create()

# Détecter les régions MSER
regions,_ = mser.detectRegions(img)

# Créer les contours convexes
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]

# Créer une copie de l'image originale et la convertir en image en niveaux de gris
clone = img.copy()
clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# Dessiner les régions MSER
cv2.polylines(clone, hulls, 1, (0, 255, 0))

# Afficher l'image
cv2.namedWindow('all MSER', cv2.WINDOW_NORMAL)
cv2.imshow('all MSER', clone) 
cv2.imwrite(path_save+'/all MSER.jpg',clone)
cv2.waitKey()
cv2.destroyAllWindows()

def circularity_check(img, regions, hulls):

    # Créer des listes pour stocker les régions et les contours intéressants
    Regions = []
    Contours = []

    # Boucle sur les régions et les contours
    for index, (region, cnt) in enumerate(zip(regions, hulls)):
        area = cv2. contourArea(cnt) # Calcul de l'aire des contours
        perimeter = cv2.arcLength(cnt,True) # Calcul du périmètre des contours
        circularity = 4*np.pi*area/perimeter**2 # Calcul de la circularité
        if circularity > 0.93:
            if(len(cnt) < 14):
                Regions.append(region) # Régions intéressantes
                Contours.append(cnt) # Contours intéressants

    # Créer une copie de l'image originale et la convertir en image en niveaux de gris
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Dessiner les régions MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))
    cv2.namedWindow('circular MSER', cv2.WINDOW_NORMAL)
    cv2.imshow('circular MSER', clone)
    cv2.imwrite(path_save+"/circular MSER.jpg",clone)
    cv2.waitKey()
    cv2.destroyAllWindows()

    return Regions, Contours, clone


Regions_circ, Contours_circ, clone_circ = circularity_check(img, regions, hulls)

Eliminer certaines régions qui ne respecte pas une intensité donnée

In [21]:
def intensity_check(img, regions, hulls):

    # Créez des listes pour stocker les régions et les contours intéressants
    Regions = []
    Contours = []
    Centers = []

    # Boucle sur les régions et les contours
    for index, (region, cnt) in enumerate(zip(regions, hulls)):
        M = cv2.moments(cnt) # moments
        center = int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]) # centroid (Cx, Cy)
        bright = img[center[1],center[0]] # luminosité au centre
        bright_mean = np.mean([img[p[1],p[0]] for p in region]) # luminosité moyenne
        if bright >= 0:
            Regions.append(region) # régions intéressantes
            Contours.append(cnt) # contours intéressants
            Centers.append(center) # centres intéressants

    # Créer une copie de l'image originale et la convertir en image en niveaux de gris
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Dessiner les régions MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))

    cv2.namedWindow('selected MSER from intensity', cv2.WINDOW_NORMAL)
    cv2.imshow('selected MSER from intensity',clone)
    cv2.imwrite(path_save+'/selected MSER from intensity.jpg',clone)
    cv2.waitKey()
    cv2.destroyAllWindows()

    return Regions, Contours, Centers, clone

Regions_int, Contours_int, Centers_int, clone_int = intensity_check(img, Regions_circ, Contours_circ)

Eliminer les régions maximales stables en double, en conservant uniquement les régions uniques

In [22]:
def duplicated_check(img, regions, cnt, centers):
    Box = []
    Box = [cv2.boundingRect(i) for i in cnt] # Créer une boîte englobante pour chaque contour
    B = np.array(Box) # Convertir la liste en tableau numpy
    # Suppression des boîtes englobantes dupliquées
    remove = np.zeros(B.shape[0], dtype=bool) # Créer un tableau booléen pour supprimer les boîtes englobantes dupliquées

    atol = 5 # tolérance
    
    # Boucle sur les boîtes englobantes
    for i in range(B.shape[0]):

        # Regardez si une boîte est similaire à une autre des deux côtés
        similar = np.isclose(B[i, :], B[(i + 1):, :], atol = atol)
        equals = np.all(similar, axis=1)
        
        # Supprimer les boîtes englobantes dupliquées
        remove[(i + 1):] = np.logical_or(remove[(i + 1):], equals)

    index = np.where(np.logical_not(remove))
    Bbox = B[np.logical_not(remove)]
    
    # Sélectionner les régions, les contours et les centres correspondants
    Regions = []
    Contours = []
    Centers = []

    Contours = [cnt[p] for p in index[0]]
    Regions = [regions[p] for p in index[0]]
    Centers = [centers[p] for p in index[0]]

    # Créer une copie de l'image originale et la convertir en image en niveaux de gris
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Dessiner les régions MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))
    cv2.namedWindow('duplicated MSER suppression', cv2.WINDOW_NORMAL)
    cv2.imshow('duplicated MSER suppression',clone)
    cv2.imwrite(path_save+'/duplicated MSER suppression.jpg',clone)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return Regions, Contours, Centers, Bbox, clone

Regions, Contours, Centers, Bbox, clone_sel = duplicated_check(img,Regions_int,Contours_int,Centers_int)

Evaluer les centroids moyens des régions détectées en calculant la moyenne des centroids des régions dont les centres sont contenus dans les mêmes régions, puis trie et stocke ces centroids uniques.

In [23]:
result_centroid = []
temp = []
for ind in Bbox:
    temp = []
    corner_min = np.array((ind[0],ind[1]))
    corner_max = np.array((ind[0]+ind[2],ind[1]+ind[3]))

    for centroid in Centers:
        if ((np.array(centroid)<corner_max).all() and (np.array(centroid)>corner_min).all()):
            temp.append(centroid)
    result_centroid.append(tuple(np.mean(temp,axis=0,dtype = int)))

result_centroid = list(set(result_centroid))
result_centroid = sorted(result_centroid, key=lambda x: x[1], reverse=True)

Ajouter les numéros aux centroids moyens des régions MSER séléctionnées et dessiner un rectangle autour de chaque centroid.

In [24]:
# Créez une copie de l'image originale et convertissez-la en image en niveaux de gris
clone1 = img.copy()
clone1 = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# Créer une police pour le texte
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 255)
thickness = 2
nombre=0

# Mettre en évidence les régions MSER sélectionnées
for i,c in enumerate(result_centroid):
    clone1 = cv2.rectangle(clone1, (c[0]-10,c[1]-10), (c[0]+10,c[1]+10), (0, 255, 255), thickness)
    nombre+=1
    number = str(i+1)
    cv2.putText(clone1,number,c,font,fontScale,color,thickness)
    
# Afficher l'image
cv2.polylines(clone1, Contours, 1, (0, 255, 0))
cv2.namedWindow('selected MSER', cv2.WINDOW_NORMAL)
cv2.imshow('selected MSER',clone1)
cv2.imwrite(path_save+'/selected MSER.jpg',clone1)
cv2.waitKey()
cv2.destroyAllWindows()